Obtener el fichero usando la librería requests

In [134]:
import requests
import bz2

url_inmuebles = 'https://gist.github.com/victorMaestro1994/3dfb3628c19c10ef117e6a5328b02637/raw/ec628696a6e2351e7ba7b3c1cd3023f0984eb53d/data.csv.bz2'
url_precios = 'https://gist.github.com/victorMaestro1994/3dfb3628c19c10ef117e6a5328b02637/raw/ec628696a6e2351e7ba7b3c1cd3023f0984eb53d/prices.csv.bz2'

#Obtención de la url

response = requests.get(url_inmuebles)

# Descomprimimos el contenido de la respuesta

descomprimido = bz2.decompress(response.content)

# Guardamos el contenido descomprimido en un archivo CSV

with open('inmuebles.csv', 'wb') as f:
    f.write(descomprimido)

#Repetimos el mismo proceso con la url de precios

response = requests.get(url_precios)
descomprimido=bz2.decompress(response.content)
with open('prices.csv', 'wb') as f:
    f.write(descomprimido)


Usando línea de comandos importar los datos de dichos ficheros en una base de datos que resida en un fichero SQLite.

In [135]:
import sqlite3

con_idealista = sqlite3.connect('idealista.db')
cur_idealista = con_idealista.cursor()



1. Número de inmuebles en alquiler que tiene cada cliente profesional por código postal

In [136]:

query = """ 
SELECT customer_id, zipcode, COUNT(*) as num_inmuebles
FROM inmuebles
WHERE is_professional = 1 and transaction_type = 'rent'
GROUP BY customer_id, zipcode;
"""

cur_idealista.execute(query)
resultados = cur_idealista.fetchall()

for resultado in resultados:
    print(f"Cliente {resultado[0]} en el código postal {resultado[1]} tiene {resultado[2]} inmuebles en alquiler.")



Cliente 393030303830303030333431 en el código postal 8950 tiene 2 inmuebles en alquiler.
Cliente 393030303830303030333431 en el código postal 8960 tiene 8 inmuebles en alquiler.
Cliente 393030303830303030333431 en el código postal 8970 tiene 1 inmuebles en alquiler.
Cliente 393030313430303030323137 en el código postal 8001 tiene 1 inmuebles en alquiler.
Cliente 393030313430303030323137 en el código postal 8003 tiene 1 inmuebles en alquiler.
Cliente 393030313430303030323137 en el código postal 8009 tiene 2 inmuebles en alquiler.
Cliente 393030313430303030323137 en el código postal 8011 tiene 1 inmuebles en alquiler.
Cliente 393030313430303030323137 en el código postal 8013 tiene 1 inmuebles en alquiler.
Cliente 393030313430303030323137 en el código postal 8014 tiene 2 inmuebles en alquiler.
Cliente 393030313430303030323137 en el código postal 8017 tiene 1 inmuebles en alquiler.
Cliente 393030313430303030323137 en el código postal 8019 tiene 1 inmuebles en alquiler.
Cliente 3930303134303

2.Obtén los siguientes códigos postales de la ciudad de Barcelona:

In [137]:
query = """
SELECT DISTINCT zipcode
FROM inmuebles
WHERE city = 'Barcelona Capital';
"""

cur_idealista.execute(query)
codigos_postales_barcelona = cur_idealista.fetchall()

print("Códigos postales de Barcelona:", codigos_postales_barcelona)


Códigos postales de Barcelona: [('8038',), ('8034',), ('8017',), ('8029',), ('8024',), ('8026',), ('8005',), ('8002',), ('8027',), ('8023',), ('8022',), ('8007',), ('8014',), ('8033',), ('8013',), ('8019',), ('8015',), ('8009',), ('8004',), ('8041',), ('8025',), ('8035',), ('8011',), ('8006',), ('8001',), ('8031',), ('8016',), ('8030',), ('8021',), ('8032',), ('8008',), ('8003',), ('8028',), ('8018',), ('8020',), ('8010',), ('8037',), ('8036',), ('8640',), ('8012',), ('8042',), ('8196',), ('8039',), ('8397',), ('8040',), ('8918',), ('17460',), ('832',), ('99999',), ('0',), ('8200',), ('8930',), ('8480',), ('8908',), ('8203',), ('8902',), ('8903',), ('8775',), ('8680',), ('8401',), ('8185',), ('8240',), ('31004',), ('8907',), ('8440',), ('8402',), ('8922',), ('8870',), ('8490',), ('8780',), ('8211',), ('8736',), ('8757',), ('8290',), ('8445',), ('8210',), ('43881',), ('8820',), ('8051',), ('9009',), ('43700',), ('8731',), ('88010',), ('8109',), ('8390',), ('8150',), ('8786',), ('8254',)

3.Código postal del precio por metro cuadrado más barato

In [138]:

query = """
SELECT zipcode
FROM inmuebles LEFT JOIN prices ON inmuebles.ad_id=prices.id
ORDER BY price/area 
LIMIT 1
"""

cur_idealista.execute(query)
codigo_mas_barato = cur_idealista.fetchall()

print("Código postal con el metro cuadrado más barato:", codigo_mas_barato)


Código postal con el metro cuadrado más barato: [('8034',)]


4.Código postal del precio por metro cuadrado más caro

In [139]:
query = """
SELECT zipcode
FROM inmuebles LEFT JOIN prices ON inmuebles.ad_id=prices.id
ORDER BY price/area DESC
LIMIT 1
"""

cur_idealista.execute(query)
codigo_mas_caro = cur_idealista.fetchall()

print("Código postal con el metro cuadrado más caro:", codigo_mas_caro)


Código postal con el metro cuadrado más caro: [('8027',)]


5.Cliente profesional que tiene la cartera de inmuebles más envejecida

In [140]:
# En esta query podemos inerpretarlo en dos sentidos, 
# el primero con la media de inmuebles más antiguos
# o cliente que tenga el anuncion con más tiempo publicado


query = """
SELECT customer_id
FROM inmuebles 
WHERE is_professional=1
GROUP BY customer_id
ORDER BY AVG(date_added)
LIMIT 1
"""

cur_idealista.execute(query)
cliente = cur_idealista.fetchall()



print("Cliente con la cartera (media) más envejecida:", cliente)

query = """
SELECT customer_id
FROM inmuebles 
WHERE is_professional=1
ORDER BY date_added
LIMIT 1
"""

cur_idealista.execute(query)
cliente = cur_idealista.fetchall()



print("Cliente con el inmueble más envejecido:", cliente)


Cliente con la cartera (media) más envejecida: [('39323032373531313531353331',)]
Cliente con el inmueble más envejecido: [('393433303033303635343733',)]


6.En qué ciudad se encuentra el terreno urbanizable más caro?

In [141]:
query = """
SELECT city
FROM inmuebles LEFT JOIN prices ON inmuebles.ad_id=prices.id
WHERE subtypology = 'Terreno Urbanizable'
ORDER BY price/area_land DESC
LIMIT 1
"""

cur_idealista.execute(query)
terreno_mas_caro = cur_idealista.fetchall()

print(f"Ciudad con el terreno urbanizable más caro:", terreno_mas_caro)


Ciudad con el terreno urbanizable más caro: [('Barcelona Capital',)]


 Realiza los mismos ejercicios del apartado 4 utilizando la librería pandas

In [142]:
import pandas as pd

1.Número de inmuebles en alquiler que tiene cada cliente profesional por código postal

In [143]:

resultados = df_inmuebles[df_inmuebles['is_professional'] == 1].groupby(['customer_id', 'zipcode']).size().reset_index(name='num_inmuebles')

print(resultados)

                    customer_id zipcode  num_inmuebles
0      353030313830303037353437    8398              1
1      353030313830303230343630    8203              1
2      363030313430303030313039    8013              1
3      363030313430303030313039    8930              1
4      363030313430303030323932    8758              1
...                         ...     ...            ...
38487  393433303033313033373236    8358             15
38488  393433303033313033373236    8360              1
38489  393433303033313033373236    8392              9
38490  393433303033313033373236    8394              7
38491  393433303033313033373236    8396              1

[38492 rows x 3 columns]


2.Obtén los siguientes códigos postales de la ciudad de Barcelona:

In [144]:
codigos_postales_barcelona = df_inmuebles[df_inmuebles['city'] == 'Barcelona Capital']['zipcode'].unique()
print(df_inmuebles)
display(codigos_postales_barcelona)

          ad_id                 customer_id                       city  \
0       5156564  39323032373531333936323630          Barcelona Capital   
1       5156565  39323032373532323235313938          Barcelona Capital   
2       5156566  39323032373534303438333537          Barcelona Capital   
3       5156567  39323032373531373435393038                    Dosrius   
4       5156568  39323032373534353337323338          Barcelona Capital   
...         ...                         ...                        ...   
122369  5278933    393433303033303138363338              Pineda de Mar   
122370  5278934  39323032373531333735313830        Olesa de Montserrat   
122371  5278935  39323032373632343634333535                 Montesquiu   
122372  5278936  39323032373631383234353237  L'Hospitalet de Llobregat   
122373  5278937  39323032373633373632363233          Montcada i Reixac   

        typology          subtypology transaction_type zipcode  bedrooms  \
0       Vivienda               Dúpl

array(['8038', '8034', '8017', '8029', '8024', '8026', '8005', '8002',
       '8027', '8023', '8022', '8007', '8014', '8033', '8013', '8019',
       '8015', '8009', '8004', '8041', '8025', '8035', '8011', '8006',
       '8001', '8031', '8016', '8030', '8021', '8032', '8008', '8003',
       '8028', '8018', '8020', '8010', '8037', '8036', '8640', '8012',
       '8042', '8196', '8039', '8397', '8040', '8918', '17460', '832',
       '99999', '0', '8200', '8930', '8480', '8908', '8203', '8902',
       '8903', '8775', '8680', '8401', '8185', '8240', '31004', '8907',
       '8440', '8402', '8922', '8870', '8490', '8780', '8211', '8736',
       8006, 8007, 8027, 8034, 8029, 8014, 8008, 8028, 8001, 8025, 8013,
       8022, 8041, 8015, 8016, 8002, 8037, 8003, 8019, 8018, 8017, 8020,
       8024, 8026, 8042, 8035, 8030, 8012, 8010, 8031, 8004, 8009, 8005,
       8021, 8011, 8032, 8023, 8038, 8036, 8033, 8757, 8040, 8902, 8039,
       8930, 99999, 8397, 8290, 8402, 8903, 8445, 8210, 8680, 8401, 43

3.Código postal del precio por metro cuadrado más barato

In [148]:
# Unimos los dataframes
df_join = pd.merge(df_inmuebles, df_prices, left_on='ad_id', right_on='id', how='left')


df_join['price_area'] = df_join['price']/df_join['area']
codigo_mas_barato = df_join.sort_values(by='price_area')


print("Código postal con el metro cuadrado más barato:", codigo_mas_barato['zipcode'].values[0])


Código postal con el metro cuadrado más barato: 8380


4.Código postal del precio por metro cuadrado más caro

In [146]:
# Utilizamos el dataframe con el join anterior pero reordenamos los resultados
codigo_mas_caro = df_join.sort_values(by='price_area', ascending=False)

print("Código postal con el metro cuadrado más caro:", codigo_mas_caro['zipcode'].values[0])


Código postal con el metro cuadrado más caro: 8016


5. Cliente profesional que tiene la cartera de inmuebles más envejecida

In [150]:
#Igual que en el caso de las sentencias SQL vamos a estudiar las dos posibles variaciones del enunciado

# Nos aseguramos de que la columna 'date_added' esté en formato de fecha y hora
df_inmuebles['date_added'] = pd.to_datetime(df_inmuebles['date_added'])

# Cliente con la cartera (media) más envejecida
cliente_cartera_mas_envejecida = df_inmuebles[df_inmuebles['is_professional'] == 1].groupby('customer_id')['date_added'].mean().idxmax()

print("Cliente con la cartera (media) más envejecida:", cliente_cartera_mas_envejecida)

# Cliente con el inmueble más envejecido
cliente_inmueble_mas_envejecido = df_inmuebles[df_inmuebles['is_professional'] == 1].sort_values(by='date_added').iloc[0]['customer_id']

print("Cliente con el inmueble más envejecido:", cliente_inmueble_mas_envejecido)

Cliente con la cartera (media) más envejecida: 39323032373633363535303138
Cliente con el inmueble más envejecido: 393433303033303635343733


6. En qué ciudad se encuentra el terreno urbanizable más caro?

In [151]:
# Primero obtenemos un dataframe con solo aquellos inmuebles de tipo "Terreno Urbanizable"
df_terrenos = df_inmuebles[df_inmuebles['subtypology'] == 'Terreno Urbanizable']

# Unimos nuestro nuevo dataframe con el de prices
df_join = pd.merge(df_terrenos, df_prices, left_on='ad_id', right_on='id', how='left')

# Utilizamos el dataframe en el que unimos los terrenos urbanizables con prices
df_join['price_area_land'] = df_join['price'] / df_join['area_land']

# Obtener la ciudad con el terreno urbanizable más caro
terreno_mas_caro = df_join[df_join['subtypology'] == 'Terreno Urbanizable'].sort_values(by='price_area_land', ascending=False).iloc[0]['city']

print("Ciudad con el terreno urbanizable más caro:", terreno_mas_caro)

Ciudad con el terreno urbanizable más caro: Dosrius


Un inversor quiere saber por código postal qué propiedades son interesantes. Añade al dataframe de pandas las siguientes columnas:


In [154]:
# Definimos umbrales para clasificación ( se toman valores estimados)
over_valued = 1.5
under_valued = 0.5

#Utilizaremos nuestro dataframe de union con prices

df_join = pd.merge(df_inmuebles, df_prices, left_on='ad_id', right_on='id', how='left')
df_join['price_area']=df_join['price']/df_join['area']

# Agregmos las columnas 'over_valued', 'normal', y 'under_valued' a nuesftro dataframe
df_join['over_valued'] = (df_join['price_area'] > over_valued).astype(int)
df_join['normal'] = ((df_join['price_area'] >= under_valued) & (df_join['price_area'] <= over_valued)).astype(int)
df_join['under_valued'] = (df_join['price_area'] < under_valued).astype(int)


codigo_postal_inversion = df_join.groupby('zipcode')['under_valued'].sum().idxmax()

print("Código postal con el mayor número de propiedades interesantes:", codigo_postal_inversion)

Código postal con el mayor número de propiedades interesantes: 8830


Guarda el nuevo dataframe del anterior apartado en un fichero json comprimido con gzip

In [161]:
import gzip

df_to_json=df_join.groupby('zipcode')['under_valued']
ruta = "./dataframeresult"

# Guardar el DataFrame en un archivo JSON
df_join.to_json(ruta, orient='records', lines=True, compression='gzip')